# ساخت یک مدل طبقه‌بندی: غذاهای خوشمزه آسیایی و هندی


## طبقه‌بندی‌کننده‌های آشپزی ۲

در این درس دوم از طبقه‌بندی، ما به بررسی `روش‌های بیشتری` برای طبقه‌بندی داده‌های دسته‌بندی‌شده خواهیم پرداخت. همچنین درباره پیامدهای انتخاب یک طبقه‌بندی‌کننده نسبت به دیگری یاد خواهیم گرفت.

### [**آزمون پیش از درس**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **پیش‌نیاز**

فرض ما این است که شما درس‌های قبلی را گذرانده‌اید، زیرا برخی از مفاهیمی که قبلاً یاد گرفته‌ایم را ادامه خواهیم داد.

برای این درس، به بسته‌های زیر نیاز داریم:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) یک [مجموعه از بسته‌های R](https://www.tidyverse.org/packages) است که برای سریع‌تر، آسان‌تر و لذت‌بخش‌تر کردن علم داده طراحی شده است!

-   `tidymodels`: چارچوب [tidymodels](https://www.tidymodels.org/) یک [مجموعه از بسته‌ها](https://www.tidymodels.org/packages/) برای مدل‌سازی و یادگیری ماشین است.

-   `themis`: بسته [themis](https://themis.tidymodels.org/) مراحل اضافی برای مقابله با داده‌های نامتعادل را فراهم می‌کند.

می‌توانید این بسته‌ها را با دستور زیر نصب کنید:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

همچنین، اسکریپت زیر بررسی می‌کند که آیا بسته‌های مورد نیاز برای تکمیل این ماژول را دارید یا خیر و در صورت نبود، آن‌ها را برای شما نصب می‌کند.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. یک نقشه طبقه‌بندی**

در [درس قبلی](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1)، سعی کردیم به این پرسش پاسخ دهیم: چگونه بین مدل‌های مختلف انتخاب کنیم؟ تا حد زیادی این انتخاب به ویژگی‌های داده و نوع مسئله‌ای که می‌خواهیم حل کنیم بستگی دارد (برای مثال طبقه‌بندی یا رگرسیون؟)

قبلاً درباره گزینه‌های مختلفی که هنگام طبقه‌بندی داده‌ها در اختیار دارید، با استفاده از برگه تقلب مایکروسافت یاد گرفتیم. چارچوب یادگیری ماشین پایتون، Scikit-learn، یک برگه تقلب مشابه اما با جزئیات بیشتر ارائه می‌دهد که می‌تواند به شما در محدود کردن انتخاب تخمین‌گرها (اصطلاح دیگری برای طبقه‌بندها) کمک کند:

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> نکته: [این نقشه را به صورت آنلاین مشاهده کنید](https://scikit-learn.org/stable/tutorial/machine_learning_map/) و با کلیک روی مسیرها مستندات را بخوانید.
>
> سایت [مرجع Tidymodels](https://www.tidymodels.org/find/parsnip/#models) نیز مستندات بسیار خوبی درباره انواع مختلف مدل‌ها ارائه می‌دهد.

### **برنامه** 🗺️

این نقشه زمانی که درک واضحی از داده‌های خود داشته باشید بسیار مفید است، زیرا می‌توانید با دنبال کردن مسیرهای آن به یک تصمیم برسید:

-   ما بیش از ۵۰ نمونه داریم

-   می‌خواهیم یک دسته‌بندی را پیش‌بینی کنیم

-   داده‌های برچسب‌دار داریم

-   کمتر از ۱۰۰ هزار نمونه داریم

-   ✨ می‌توانیم یک Linear SVC انتخاب کنیم

-   اگر این کار نکرد، چون داده‌های ما عددی هستند

    -   می‌توانیم یک ✨ KNeighbors Classifier امتحان کنیم

        -   اگر این هم جواب نداد، ✨ SVC و ✨ Ensemble Classifiers را امتحان کنید

این یک مسیر بسیار مفید برای دنبال کردن است. حالا، بیایید با استفاده از چارچوب مدل‌سازی [tidymodels](https://www.tidymodels.org/) شروع کنیم: مجموعه‌ای منسجم و انعطاف‌پذیر از بسته‌های R که برای تشویق به شیوه‌های آماری خوب توسعه داده شده‌اند 😊.

## ۲. تقسیم داده‌ها و مدیریت مجموعه داده‌های نامتوازن

از درس‌های قبلی یاد گرفتیم که مجموعه‌ای از مواد اولیه مشترک در میان غذاهای مختلف وجود داشت. همچنین، توزیع تعداد غذاها بسیار نابرابر بود.

ما این موارد را به این صورت مدیریت می‌کنیم:

-   حذف مواد اولیه‌ای که بیشترین اشتباه را بین غذاهای مختلف ایجاد می‌کنند، با استفاده از `dplyr::select()`.

-   استفاده از یک `recipe` که داده‌ها را پیش‌پردازش می‌کند تا با اعمال یک الگوریتم `over-sampling` برای مدل‌سازی آماده شوند.

ما قبلاً این موارد را در درس قبلی بررسی کردیم، بنابراین این کار باید آسان باشد 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### مقابله با داده‌های نامتوازن

داده‌های نامتوازن اغلب تأثیرات منفی بر عملکرد مدل دارند. بسیاری از مدل‌ها زمانی بهترین عملکرد را دارند که تعداد مشاهدات برابر باشد و به همین دلیل با داده‌های نامتوازن دچار مشکل می‌شوند.

دو روش اصلی برای مقابله با مجموعه داده‌های نامتوازن وجود دارد:

-   اضافه کردن مشاهدات به کلاس اقلیت: `Over-sampling`، به عنوان مثال استفاده از الگوریتم SMOTE که به صورت مصنوعی نمونه‌های جدیدی از کلاس اقلیت را با استفاده از نزدیک‌ترین همسایگان این موارد تولید می‌کند.

-   حذف مشاهدات از کلاس اکثریت: `Under-sampling`

در درس قبلی، نشان دادیم که چگونه می‌توان با استفاده از یک `recipe` با مجموعه داده‌های نامتوازن برخورد کرد. یک recipe را می‌توان به عنوان یک نقشه راه در نظر گرفت که توضیح می‌دهد چه مراحلی باید روی یک مجموعه داده اعمال شود تا برای تحلیل داده آماده شود. در مورد ما، می‌خواهیم توزیع برابری در تعداد غذاهای مختلف برای `training set` خود داشته باشیم. بیایید مستقیماً وارد موضوع شویم.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

حالا آماده‌ایم تا مدل‌ها را آموزش دهیم! 👩‍💻👨‍💻

## ۳. فراتر از مدل‌های رگرسیون چندجمله‌ای

در درس قبلی، به مدل‌های رگرسیون چندجمله‌ای پرداختیم. بیایید مدل‌های انعطاف‌پذیرتری برای طبقه‌بندی بررسی کنیم.

### ماشین‌های بردار پشتیبان

در زمینه طبقه‌بندی، `ماشین‌های بردار پشتیبان` یک تکنیک یادگیری ماشین است که تلاش می‌کند یک *ابرصفحه* پیدا کند که به بهترین شکل ممکن کلاس‌ها را از هم جدا کند. بیایید یک مثال ساده را بررسی کنیم:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ کلاس‌ها را جدا نمی‌کند. H2~ کلاس‌ها را جدا می‌کند، اما فقط با یک فاصله کوچک. H3~ کلاس‌ها را با بیشترین فاصله جدا می‌کند.

#### طبقه‌بند خطی بردار پشتیبان

خوشه‌بندی بردار پشتیبان (SVC) یکی از اعضای خانواده تکنیک‌های یادگیری ماشین بردار پشتیبان است. در SVC، ابرصفحه‌ای انتخاب می‌شود که بتواند `بیشتر` مشاهدات آموزشی را به درستی جدا کند، اما ممکن است `چند مشاهده` را اشتباه طبقه‌بندی کند. با اجازه دادن به برخی نقاط برای قرار گرفتن در سمت اشتباه، SVM نسبت به داده‌های پرت مقاوم‌تر می‌شود و در نتیجه تعمیم بهتری به داده‌های جدید دارد. پارامتری که این تخطی را تنظیم می‌کند، به نام `cost` شناخته می‌شود که مقدار پیش‌فرض آن 1 است (به `help("svm_poly")` مراجعه کنید).

بیایید یک SVC خطی ایجاد کنیم با تنظیم `degree = 1` در یک مدل SVM چندجمله‌ای.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

حالا که مراحل پیش‌پردازش و مشخصات مدل را در قالب یک *جریان کاری* ثبت کرده‌ایم، می‌توانیم به آموزش مدل SVC خطی بپردازیم و در همین حین نتایج را ارزیابی کنیم. برای معیارهای عملکرد، بیایید یک مجموعه معیار ایجاد کنیم که موارد زیر را ارزیابی کند: `دقت`، `حساسیت`، `ارزش پیش‌بینی مثبت` و `F Measure`.

> تابع `augment()` ستون(هایی) برای پیش‌بینی‌ها به داده‌های داده‌شده اضافه می‌کند.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### ماشین بردار پشتیبان

ماشین بردار پشتیبان (SVM) نسخه‌ای توسعه‌یافته از طبقه‌بند بردار پشتیبان است که برای ایجاد مرز غیرخطی بین کلاس‌ها طراحی شده است. به طور کلی، SVMها از *ترفند کرنل* برای گسترش فضای ویژگی استفاده می‌کنند تا بتوانند روابط غیرخطی بین کلاس‌ها را تطبیق دهند. یکی از توابع کرنل محبوب و بسیار انعطاف‌پذیری که توسط SVMها استفاده می‌شود، *تابع پایه شعاعی* است. بیایید ببینیم این روش چگونه روی داده‌های ما عمل می‌کند.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

خیلی بهتر 🤩!

> ✅ لطفاً ببینید:
>
> -   [*ماشین‌های بردار پشتیبان*](https://bradleyboehmke.github.io/HOML/svm.html)، یادگیری ماشین عملی با R
>
> -   [*ماشین‌های بردار پشتیبان*](https://www.statlearning.com/)، مقدمه‌ای بر یادگیری آماری با کاربردهایی در R
>
> برای مطالعه بیشتر.

### طبقه‌بندهای نزدیک‌ترین همسایه

الگوریتم *K*-نزدیک‌ترین همسایه (KNN) الگوریتمی است که در آن هر مشاهده بر اساس *شباهت* آن به سایر مشاهدات پیش‌بینی می‌شود.

بیایید یکی از این الگوریتم‌ها را روی داده‌های خود اعمال کنیم.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

به نظر می‌رسد که این مدل عملکرد خوبی ندارد. احتمالاً تغییر پارامترهای مدل (به کمک `help("nearest_neighbor")`) می‌تواند عملکرد مدل را بهبود بخشد. حتماً آن را امتحان کنید.

> ✅ لطفاً ببینید:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> برای یادگیری بیشتر درباره طبقه‌بندهای *K*-نزدیک‌ترین همسایه‌ها.

### طبقه‌بندهای ترکیبی

الگوریتم‌های ترکیبی با ترکیب چندین مدل پایه برای ایجاد یک مدل بهینه عمل می‌کنند، به یکی از روش‌های زیر:

`bagging`: اعمال یک *تابع میانگین‌گیری* بر مجموعه‌ای از مدل‌های پایه

`boosting`: ساخت یک دنباله از مدل‌ها که بر اساس یکدیگر ساخته می‌شوند تا عملکرد پیش‌بینی بهبود یابد.

بیایید با امتحان کردن یک مدل جنگل تصادفی شروع کنیم، که مجموعه بزرگی از درخت‌های تصمیم‌گیری می‌سازد و سپس یک تابع میانگین‌گیری را اعمال می‌کند تا یک مدل کلی بهتر ایجاد شود.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

کار عالی 👏!

بیایید همچنین با مدل Boosted Tree آزمایش کنیم.

Boosted Tree یک روش ترکیبی را تعریف می‌کند که مجموعه‌ای از درخت‌های تصمیم‌گیری متوالی ایجاد می‌کند، به‌طوری‌که هر درخت به نتایج درخت‌های قبلی وابسته است و تلاش می‌کند به‌صورت تدریجی خطا را کاهش دهد. این روش بر وزن مواردی که به‌اشتباه طبقه‌بندی شده‌اند تمرکز می‌کند و برازش مدل بعدی را تنظیم می‌کند تا این اشتباهات را اصلاح کند.

روش‌های مختلفی برای برازش این مدل وجود دارد (به `help("boost_tree")` مراجعه کنید). در این مثال، ما درخت‌های Boosted را از طریق موتور `xgboost` برازش خواهیم داد.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ لطفاً ببینید:
>
> -   [یادگیری ماشین برای دانشمندان اجتماعی](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [یادگیری ماشین عملی با R](https://bradleyboehmke.github.io/HOML/)
>
> -   [مقدمه‌ای بر یادگیری آماری با کاربردهایی در R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - مدل AdaBoost را بررسی می‌کند که جایگزین خوبی برای xgboost است.
>
> برای یادگیری بیشتر درباره طبقه‌بندهای Ensemble.

## ۴. اضافی - مقایسه چندین مدل

ما در این آزمایشگاه تعداد زیادی مدل را برازش کرده‌ایم 🙌. ایجاد تعداد زیادی جریان کاری از مجموعه‌های مختلف پیش‌پردازنده‌ها و/یا مشخصات مدل و سپس محاسبه معیارهای عملکرد به‌صورت تک‌تک می‌تواند خسته‌کننده یا دشوار باشد.

بیایید ببینیم آیا می‌توانیم این مشکل را با ایجاد یک تابع که لیستی از جریان‌های کاری را روی مجموعه آموزشی برازش می‌کند و سپس معیارهای عملکرد را بر اساس مجموعه آزمایشی بازمی‌گرداند، حل کنیم. ما از `map()` و `map_dfr()` از بسته [purrr](https://purrr.tidyverse.org/) استفاده خواهیم کرد تا توابع را روی هر عنصر در لیست اعمال کنیم.

> توابع [`map()`](https://purrr.tidyverse.org/reference/map.html) به شما این امکان را می‌دهند که بسیاری از حلقه‌های for را با کدی جایگزین کنید که هم مختصرتر و هم خواناتر است. بهترین مکان برای یادگیری درباره توابع [`map()`](https://purrr.tidyverse.org/reference/map.html) فصل [تکرار](http://r4ds.had.co.nz/iteration.html) در کتاب R برای علم داده است.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


بسته [**workflowset**](https://workflowsets.tidymodels.org/) به کاربران این امکان را می‌دهد که تعداد زیادی مدل ایجاد کرده و به‌راحتی آن‌ها را برازش کنند، اما بیشتر برای کار با تکنیک‌های بازنمونه‌گیری مانند `cross-validation` طراحی شده است، رویکردی که هنوز به آن نپرداخته‌ایم.

## **🚀چالش**

هر یک از این تکنیک‌ها تعداد زیادی پارامتر دارند که می‌توانید آن‌ها را تنظیم کنید، برای مثال `cost` در SVMها، `neighbors` در KNN، و `mtry` (پیش‌بینی‌کننده‌های انتخابی تصادفی) در جنگل تصادفی.

پارامترهای پیش‌فرض هر کدام را بررسی کنید و به این فکر کنید که تنظیم این پارامترها چه تأثیری بر کیفیت مدل خواهد داشت.

برای اطلاعات بیشتر درباره یک مدل خاص و پارامترهای آن، از دستور زیر استفاده کنید: `help("model")` مثلاً `help("rand_forest")`

> در عمل، ما معمولاً *بهترین مقادیر* را با آموزش تعداد زیادی مدل روی یک `مجموعه داده شبیه‌سازی‌شده` و اندازه‌گیری عملکرد این مدل‌ها *تخمین می‌زنیم*. این فرآیند **تنظیم** نامیده می‌شود.

### [**آزمون پس از درس**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **مرور و مطالعه شخصی**

در این درس‌ها اصطلاحات زیادی وجود دارد، بنابراین چند دقیقه وقت بگذارید و [این فهرست](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) از اصطلاحات مفید را مرور کنید!

#### سپاس ویژه از:

[`آلیسون هورست`](https://twitter.com/allison_horst/) برای خلق تصاویر شگفت‌انگیزی که R را جذاب‌تر و دوستانه‌تر کرده است. تصاویر بیشتر را در [گالری او](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM) پیدا کنید.

[Cassie Breviu](https://www.twitter.com/cassieview) و [Jen Looper](https://www.twitter.com/jenlooper) برای ایجاد نسخه اصلی پایتون این ماژول ♥️

با آرزوی یادگیری شاد،

[اریک](https://twitter.com/ericntay)، سفیر طلایی دانشجویی Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>اثر هنری از @allison_horst</figcaption>



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما برای دقت تلاش می‌کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادقتی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، ترجمه حرفه‌ای انسانی توصیه می‌شود. ما هیچ مسئولیتی در قبال سوءتفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
